TODO:
- zet annotation_info dict om in een echte Class
- encapsulate begin_anchor en end_anchor, zorg dat ze verschillend kunnen worden geimplementeerd
- voeg get_begin_identifier() en get_end_identifier() methods toe, voor uniform client gebruik
- maak mijn gebruik van begin_char_offset en end_char_offset consistent met 1. TEI casus 2. Rik's begin en end voor attendants

In [1]:
import sys
sys.path.append('../../packages')
sys.path.append('../../../elucidate-python-client')

from elucidate.client import ElucidateClient
from model import AnnotationIdentifier, ContainerIdentifier

elucidate = ElucidateClient("https://elucidate.tt.di.huc.knaw.nl/annotation")

import requests
import json
import uuid
import os
from textservice import segmentedtext
from urllib.parse import urlparse

ModuleNotFoundError: No module named 'model'

Retrieve the annotation for a specific resolution in the sample 1728 data.

In [31]:
annotation_collection = elucidate.search_by_body_id(f"urn:example:republic:{resolution_id}")
annotation_collection.total

1

In [16]:
resolution_annotation = next(annotation_collection.annotations_as_json())
resolution_annotation

{'id': 'https://elucidate.tt.di.huc.knaw.nl/annotation/w3c/republic-1728-06-19/7008b015-487e-43b9-926a-47983624c55e',
 'type': 'Annotation',
 'created': '2021-08-25T17:38:08.462651',
 'generator': {'id': 'https://github.com/knaw-huc/un-t-ann-gle',
  'type': 'Software',
  'name': 'un-t-ann-gle'},
 'body': [{'id': 'urn:example:republic:meeting-1728-06-19-session-1-resolution-17',
   'type': 'TextualBody',
   'value': 'resolution',
   'purpose': 'classifying'},
  {}],
 'target': [{'selector': {'type': 'urn:example:republic:TextAnchorSelector',
    'end': 56499,
    'start': 56378},
   'source': 'https://demorepo.tt.di.huc.knaw.nl/task/find/volume-1728/contents'},
  {'type': 'Image',
   'selector': {'type': 'FragmentSelector',
    'conformsTo': 'http://www.w3.org/TR/media-frags/',
    'value': 'xywh=345,2728,878,636'},
   'source': 'https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3783/NL-HaNA_1.01.02_3783_0287.jpg/full/,3100/0/default.jpg'},
  {'type': 'Image',
   'selector': {'type': 'Fr

In [17]:
resolution_target = resolution_annotation['target']
text_target = [t for t in resolution_target if not 'type' in t]
resource_url = text_target[0]['source']
resource_path = urlparse(resource_url).path
resource_id = os.path.split(os.path.split(resource_path)[0])[1]
resource_id

'volume-1728'

In [18]:
begin_anchor = text_target[0]['selector']['start']
end_anchor = text_target[0]['selector']['end']
print(begin_anchor)
print(end_anchor)

56378
56499


In [19]:
response = requests.get(f"http://localhost:5000/{resource_id}/segmentedtext/textgrid/{begin_anchor},{end_anchor}")

In [20]:
response.json()

{'textgrid': {'_ordered_segments': ['Ntfangen een Mifiive van den Raad:',
   '& NP van Staate , gefchreven alhier in den',
   'Hage den feventienden defer loopende',
   'maandt; houdende, dat op den een en twin',
   'tighften Mey laatftleden ter Vergaderinge',
   'van haar Hoogh Mogende was uytgebragnt',
   'ecn rapport van de Heeren haar Hoogh Mo.',
   'gende Gedeputeerden tor de faaken van de',
   'Finartie ‚ met eenige Heeren Gecommit',
   'teerden uyt den Raads van Sraate , geëxa',
   'mineert hadden haare Miflive van den fe:',
   'venden daar te vooren ; welck rapport daar',
   'toe tendeert, dat het profyt van de jegen:',
   'woordigh onder handen zijnde Generaliteyts',
   'Loteryc, en het geene van de voorige was',
   'geprofiteert , te {amen ter fomme van hon-',
   'dert feventigh duyfent guldens, geëmployeert',
   'fouden werden tot de Fortificatien, om re-',
   'denen by het rapport geallegeert ; dogh dat',
   'daar op is alsdoen, en foo veel fy weeten,',
   'tot nogh toe gee

toon text_grid als een pseudo-visuele weergave, met segment index voorafgaand aan ieder line segment

In [21]:
textgrid = segmentedtext.IndexedSegmentedText.from_json(response.json()['textgrid'])
segments = textgrid._ordered_segments

for i, segment in enumerate(segments):
    print(f"{i+textgrid.text_grid_spec['begin_offset_in_resource']}: {segment}")

56378: Ntfangen een Mifiive van den Raad:
56379: & NP van Staate , gefchreven alhier in den
56380: Hage den feventienden defer loopende
56381: maandt; houdende, dat op den een en twin
56382: tighften Mey laatftleden ter Vergaderinge
56383: van haar Hoogh Mogende was uytgebragnt
56384: ecn rapport van de Heeren haar Hoogh Mo.
56385: gende Gedeputeerden tor de faaken van de
56386: Finartie ‚ met eenige Heeren Gecommit
56387: teerden uyt den Raads van Sraate , geëxa
56388: mineert hadden haare Miflive van den fe:
56389: venden daar te vooren ; welck rapport daar
56390: toe tendeert, dat het profyt van de jegen:
56391: woordigh onder handen zijnde Generaliteyts
56392: Loteryc, en het geene van de voorige was
56393: geprofiteert , te {amen ter fomme van hon-
56394: dert feventigh duyfent guldens, geëmployeert
56395: fouden werden tot de Fortificatien, om re-
56396: denen by het rapport geallegeert ; dogh dat
56397: daar op is alsdoen, en foo veel fy weeten,
56398: tot nogh toe geen conclufi

Simuleer selectie van een stukje tekst, nl 'den Hage' op regels 1 en 2

In [22]:
begin_anchor_id = 1
end_anchor_id = 2
begin_char_offset = 39
end_char_offset = 4

print(segments[1:3])

['& NP van Staate , gefchreven alhier in den', 'Hage den feventienden defer loopende']


In [23]:
# print(segments[43][begin_offset:]+segments[44][:end_offset])
print(segments[begin_anchor_id][begin_char_offset:], end=' ')
for seg in segments[begin_anchor_id+1:end_anchor_id]:
    print(seg, end='')
print(segments[end_anchor_id][:end_char_offset])

den Hage


In [24]:
# find begin and end anchors by id
offset = textgrid.text_grid_spec['begin_offset_in_resource']

print(f'ba: {begin_anchor_id+offset} ea: {end_anchor_id+offset}')

ba: 56379 ea: 56380


In [25]:
entity_ann = {'resource_id': resource_id, 'label':'entity',\
              'begin_anchor': begin_anchor_id + offset,\
              'end_anchor': end_anchor_id + offset,\
              'id': 'annot_'+str(uuid.uuid4()), 'entity_type': 'loc', \
              'entity_text': 'Den Haag'}

entity_ann

{'resource_id': 'volume-1728',
 'label': 'entity',
 'begin_anchor': 56379,
 'end_anchor': 56380,
 'id': 'annot_7e19a2be-1b0c-4496-8583-006e2d3a4b35',
 'entity_type': 'loc',
 'entity_text': 'Den Haag'}

In [26]:
entity_ann['begin_char_offset'] = begin_char_offset
entity_ann['end_char_offset'] = end_char_offset

entity_ann

{'resource_id': 'volume-1728',
 'label': 'entity',
 'begin_anchor': 56379,
 'end_anchor': 56380,
 'id': 'annot_7e19a2be-1b0c-4496-8583-006e2d3a4b35',
 'entity_type': 'loc',
 'entity_text': 'Den Haag',
 'begin_char_offset': 39,
 'end_char_offset': 4}

Met deze info, maak benodigde body, target en custom params aan tbv elucidate create_annotation python lib call

In [27]:
body = {'id': 'urn:example:republic:' + entity_ann['id'],
      'type': 'TextualBody',
      'value': entity_ann['label'],
      'purpose': 'classifying'}

target = [{'selector': {'type': 'urn:example:republic:TextAnchorSelector',
       'end': entity_ann['begin_anchor'],
       'start': entity_ann['end_anchor']},
      'source': 'https://demorepo.tt.di.huc.knaw.nl/task/find/' + entity_ann['resource_id'] + '/contents'}]

custom = {'motivation': 'tagging',
        'entity_type': entity_ann['entity_type'],
        'entity_text': entity_ann['entity_text'],
        'begin_char_offset': entity_ann['begin_char_offset'],
        'end_char_offset': entity_ann['end_char_offset']}

print(body)
print(target)
print(custom)

{'id': 'urn:example:republic:annot_7e19a2be-1b0c-4496-8583-006e2d3a4b35', 'type': 'TextualBody', 'value': 'entity', 'purpose': 'classifying'}
[{'selector': {'type': 'urn:example:republic:TextAnchorSelector', 'end': 56379, 'start': 56380}, 'source': 'https://demorepo.tt.di.huc.knaw.nl/task/find/volume-1728/contents'}]
{'motivation': 'tagging', 'entity_type': 'loc', 'entity_text': 'Den Haag', 'begin_char_offset': 39, 'end_char_offset': 4}


Sla de nieuwe annotatie op in annotationstore

In [29]:
resolution_annotation['id']

'https://elucidate.tt.di.huc.knaw.nl/annotation/w3c/republic-1728-06-19/7008b015-487e-43b9-926a-47983624c55e'

In [33]:
annotation_identifier = AnnotationIdentifier(url=resolution_annotation['id'], etag="some etag")
container_identifier = annotation_identifier.container_identifier()
# container_identifier = ContainerIdentifier("https://elucidate.tt.di.huc.knaw.nl/annotation/w3c/republic-1728-06-19")
container_identifier

ContainerIdentifier:
  url = https://elucidate.tt.di.huc.knaw.nl/annotation/w3c/republic-1728-06-19/
  uuid = republic-1728-06-19

In [34]:
annotation_id = elucidate.create_annotation(container_id=container_identifier,body=body,target=target,custom=custom)
annotation_id.url

'https://elucidate.tt.di.huc.knaw.nl/annotation/w3c/republic-1728-06-19/ee407b2a-b090-4add-b691-a6b99f6c791e'

In [13]:
headers = {"Content-Type": "application/json"}
url = "http://localhost:5001/annotations"
response = requests.put(url, data=json.dumps(entity_ann, indent=4, cls=segmentedtext.SegmentEncoder), headers=headers)

response

<Response [200]>

TODO NEXT: as a test retrieve annotations overlapping with meeting-1728-06-01-session-1 and check if the newly created entity is included

In [14]:
url = "http://localhost:5001/volume-1728/annotations/49716,50692"
response = requests.get(url)

In [15]:
response.json()

{'annotations': [{'begin_anchor': 49716,
   'end_anchor': 49716,
   'id': 'NL-HaNA_1.01.02_3783_0258-page-515-column-1-tr-0-line-0',
   'image_coords': {'bottom': 528,
    'height': 59,
    'left': 3653,
    'right': 4155,
    'top': 469,
    'width': 502},
   'label': 'lines',
   'resource_id': 'volume-1728'},
  {'begin_anchor': 49716,
   'end_anchor': 49716,
   'id': 'NL-HaNA_1.01.02_3783_0258-page-515-column-1-tr-0-line-0',
   'image_coords': {'bottom': 528,
    'height': 59,
    'left': 3653,
    'right': 4155,
    'top': 469,
    'width': 502},
   'label': 'textregions',
   'resource_id': 'volume-1728'},
  {'begin_anchor': 49717,
   'end_anchor': 49717,
   'id': 'NL-HaNA_1.01.02_3783_0258-page-515-column-1-tr-0-line-1',
   'image_coords': {'bottom': 582,
    'height': 56,
    'left': 3831,
    'right': 3962,
    'top': 526,
    'width': 131},
   'label': 'lines',
   'resource_id': 'volume-1728'},
  {'begin_anchor': 49717,
   'end_anchor': 49717,
   'id': 'NL-HaNA_1.01.02_3783_0258